<a href="https://colab.research.google.com/github/zhouchun0105/Bitcoin-Price-and-Movement-Prediction/blob/main/Chun_and_Jinhan's_experiement_field.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bitcoin Price and its Movement Direction Prediction**
Experimental Fields

Author: Chun Zhou, BU Questrom; Jinhan Mei, University of Toronto